<a href="https://colab.research.google.com/github/sparks-baird/self-driving-lab-demo/blob/main/notebooks/6.0-olympus-benchmarking-basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install olymp gpyopt self-driving-lab-demo # olymp[all]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56 kB 2.3 MB/s 
     |████████████████████████████████| 959 kB 10.9 MB/s 
     |████████████████████████████████| 71 kB 6.6 MB/s 
  Created wheel for gpyopt: filename=GPyOpt-1.2.6-py3-none-any.whl size=83609 sha256=aff31c4e402fdc6311435938e5a0ec2658b5f5d13fb2fb0f95ba7f389d0cbb07
  Stored in directory: /root/.cache/pip/wheels/e6/fa/d1/f9652b5af79f769a0ab74dbead7c7aea9a93c6bc74543fd3ec
  Created wheel for GPy: filename=GPy-1.10.0-cp37-cp37m-linux_x86_64.whl size=2565126 sha256=6e4d5c20f79884a064f7036c6a80456492d995982d7c8ef2d65fa32d3d08c4c7
  Stored in directory: /root/.cache/pip/wheels/f7/18/28/dd1ce0192a81b71a3b086fd952511d088b21e8359ea496860a
  Created wheel for paramz: filename=paramz-0.9.5-py3-none-any.whl size=102566 sha256=dd8fd8b69fe28f00534ae92ebd8bdb094e6129e90b6372bc306893211616f0f8
  Stored in directory: /root/.cache/pip/wheels/c8/95/f5/ce

In [ ]:
import os, sys
import pickle
import time
import numpy as np
import pandas as pd
import subprocess
from uuid import uuid4 # universally unique identifier

import plotly.express as px

import olympus
from olympus.campaigns import ParameterSpace
from olympus.objects import ParameterContinuous
from olympus.campaigns import Campaign
from olympus.planners import Planner

from self_driving_lab_demo import SelfDrivingLabDemo, mqtt_observe_sensor_data
from self_driving_lab_demo.utils.plotting import line


In [ ]:
PICO_ID = "test" #@param {type:"string"}
dummy = False #@param {type:"boolean"}
SESSION_ID = str(uuid4()) # random session ID
print(f"session ID: {SESSION_ID}")


session ID: 7c2f9c57-6329-4ac7-934a-a551834c47ab


In [ ]:
#---------------
# Configuration
#---------------

model = 'Gpyopt'

if dummy:
	BUDGET = 5
	NUM_RUNS = 2
else:
	BUDGET = 20
	NUM_RUNS = 20
GOAL = 'minimize'

#-----------------------
# build olympus objects
#-----------------------

param_space = ParameterSpace()
[param_space.add(ParameterContinuous(name=name, low=0.0, high=89.0)) for name in ["R", "G", "B"]]

#-------------------
# begin experiments
#-------------------

run_ix = 0
campaigns = []

# loop through the independent runs
while run_ix < NUM_RUNS:

	sdl = SelfDrivingLabDemo(
			target_seed=run_ix,
			autoload=True, # perform target data experiment automatically
			observe_sensor_data_fn=mqtt_observe_sensor_data, # (default)
			observe_sensor_data_kwargs=dict(pico_id=PICO_ID, session_id=SESSION_ID),
	)

	print(f'\n\n STARTING SEED RUN {run_ix} on SDL-Demo using {model}\n\n')


	# build campaign and set param space
	campaign = Campaign()
	campaign.set_param_space(param_space)

	# Olympus planner
	planner = Planner(kind=model, goal=GOAL)

	# set planner param space
	planner.set_param_space(campaign.param_space)

	#-----------------
	# Begin experiment
	#-----------------

	iteration = 0

	while len(campaign.values) < BUDGET:
		print(f'\nITERATION : {iteration}\n')
		samples = planner.recommend(campaign.observations)
		print(f'SAMPLES : {samples}')
		measurement = sdl.evaluate(np.round(samples["R"]), np.round(samples["G"]), np.round(samples["B"]))["frechet"]
		campaign.add_observation(samples, measurement)

		iteration += 1
	
	campaigns.append(campaign)

	run_ix += 1



 STARTING SEED RUN 0 on SDL-Demo using Gpyopt



ITERATION : 0

SAMPLES : ParamVector(R = 4.233981973212402, G = 69.82552213353793, B = 26.62409563257783)

ITERATION : 1

SAMPLES : ParamVector(R = 0.2352895912540185, G = 84.91491854128836, B = 28.456203129800045)

ITERATION : 2

SAMPLES : ParamVector(R = 47.4734833975941, G = 42.241511908234244, B = 81.47207477513828)

ITERATION : 3

SAMPLES : ParamVector(R = 52.533020274427685, G = 13.53535491802719, B = 36.01931795383978)

ITERATION : 4

SAMPLES : ParamVector(R = 32.16166325751143, G = 41.00956515391436, B = 38.267655436247296)

ITERATION : 5

SAMPLES : ParamVector(R = 9.814082208750454, G = 12.910432120461962, B = 42.246028671655296)

ITERATION : 6

SAMPLES : ParamVector(R = 70.04999860271417, G = 14.958477309618967, B = 26.920993652174918)

ITERATION : 7

SAMPLES : ParamVector(R = 58.49324676527135, G = 39.144954538243276, B = 59.841911422558226)

ITERATION : 8

SAMPLES : ParamVector(R = 30.195181930707808, G = 78.14405695524698,

In [ ]:
obj = [camp.values for camp in campaigns]
best_obj = np.minimum.accumulate(obj, axis=1)
best_obj_mu = np.mean(best_obj, axis=0)
best_obj_std = np.std(best_obj, axis=0)
print(best_obj_mu, best_obj_std)

[9144.60305416 7325.34998895 7185.0074223  7092.29731435 6476.29169582
 6151.54166275 5749.91826037 5608.63055666 5240.06839855 5197.725807
 4749.32320901 4492.97931675 4372.36156873 4372.36156873 4248.25514335
 4095.46865168 4076.96865168 3730.51539271 3548.24969315 3540.29969315] [3198.00027859 2639.26806887 2685.64455792 2728.20903798 2244.81214595
 2382.13306123 2319.10312384 2309.74580396 2307.71587907 2339.75564649
 2067.2363088  1569.60982707 1459.38746133 1459.38746133 1523.45280105
 1344.46938348 1360.40839942 1297.06640336 1151.59307702 1157.46521877]


In [ ]:
fig = line(x=range(1,best_obj.shape[1]+1), y=best_obj_mu, error_y=best_obj_std, error_y_mode="band")
fig.update_layout(xaxis_title="iteration", yaxis_title="error")

## Example Output
![](https://github.com/sparks-baird/self-driving-lab-demo/blob/main/notebooks/olympus-benchmarking-basic.png?raw=1)